In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
df=pd.read_csv('Churn_Modelling.csv')
df=df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

df.head()
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df
df['Geography'].unique()
from sklearn.preprocessing import OneHotEncoder
one_encoder_geography=OneHotEncoder()
geo_encoder=one_encoder_geography.fit_transform(df[['Geography']])

geo_encoder=geo_encoder.toarray()

one_encoder_geography.get_feature_names_out(['Geography'])
geo_df=pd.DataFrame(geo_encoder,columns=one_encoder_geography.get_feature_names_out(['Geography']))
df=pd.concat([df.drop('Geography'
,axis=1),geo_df],axis=1)
df.head()

df.head()
X = df.drop('Exited', axis=1)
y = df['Exited']

feature_columns = X.columns   # ✅ SAVE THIS

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)





In [ ]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geography.pkl','wb') as  file:
    pickle.dump(one_encoder_geography,file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [3]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],))) 
    
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    
    return model
    

In [4]:
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [5]:
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}

In [ ]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(X_train,y_train)

print("Best:%f using %s"%(grid_result.best_score_,grid_result.best_params_))